In [216]:
constraints=["-1 -2 5 0",
             "-5 1 0",
             "-5 2 0",
             "-3 6 0",
             "-6 3 0",
             "9 0",
]

In [217]:
from copy import deepcopy
import random

In [244]:
constraintDict={}
literalsDict={}
i=0
for clause in constraints:
    if i not in constraintDict.keys():
        constraintDict[i]={}
        
    for literal in clause.split():
        lit=int(literal)
        if lit!=0:
            constraintDict[i][lit]=False
            if lit not in literalsDict.keys():
                literalsDict[lit]=[i]
            else:
                literalsDict[lit].append(i)
    i+=1
# print(constraintDict)
print(literalsDict)

{-1: [0], -2: [0], 5: [0], -5: [1, 2], 1: [1], 2: [2], -3: [3], 6: [3], -6: [4], 3: [4], 9: [5]}


In [220]:
hallo={}
hallo[1]=5
del hallo[1]
print(len(hallo))

0


In [293]:
assignments={}
def DP(constraints,literalsDict):
#     print(len(constraints))
    if len(constraints)==0:
#         print("satisfiable solution")
        return True
   
    if emptyClause(constraints):
#         print("no satisfiability")
        return False
    #maybe do it one time beforehand
    hasFound,constraints=tautology(constraints)
    if hasFound:
#         print("taut")
        DP(constraints,literalsDict)
        
    hasFound,constraints=unitCl(constraints,literalsDict)
    if hasFound:
#         print("unit")
        DP(constraints,literalsDict)
        
    hasFound,constraints=pureLit(constraints,literalsDict)
    if hasFound:
#         print("pure lit")
        DP(constraints,literalsDict)
    if len(constraints)==0:
#         print("satisfiable solution")
        return True
                
    reducedConstraints,literal=pickRandomLiteral(constraints,literalsDict,None)
#     print(len(constraints),len(reducedConstraints))
    if DP(reducedConstraints,literalsDict)==True:
#         print("satisfiable solution")
        return True
    else:
        print(literal)
        reducedConstraints,literal=pickRandomLiteral(constraints,literalsDict,-1*literal)
        print(len(constraints),len(reducedConstraints))
        DP(reducedConstraints,literalsDict)

DP(constraintDict,literalsDict)

True

In [224]:
def emptyClause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
    return False

In [225]:
constraintDict.keys()

dict_keys([0, 1, 2, 3, 4, 5])

In [226]:
def tautology(constr):
    constraints=deepcopy(constr)

    foundOne=False
    delClauses=[]
    for clause in constraints.keys():
        literals=list(constraints[clause].keys())
        counterLiterals=[-x for x in literals]
        
        for i in literals:
            for j in counterLiterals:
                if i==j:
                    delClauses.append(clause)
                    foundOne=True
                    break
    for clause in delClauses:
        try:
            del constraints[clause]
        except:
            continue
                  
    return foundOne,constraints
tautology(constraintDict)     

(False,
 {0: {-1: False, -2: False, 5: False},
  1: {-5: False, 1: False},
  2: {-5: False, 2: False},
  3: {-3: False, 6: False},
  4: {-6: False, 3: False},
  5: {9: False}})

In [227]:
def unitCl(constr,literalsDict):
    constraints=deepcopy(constr)
    literals=[]
    delClauses=[]
    for clause in constraints.keys():
        if len(constraints[clause].keys())==1:
            literal=list(constraints[clause].keys())
            for i in literal:
                literals.append(i)
            delClauses.append(clause)
#     print(literals)
    for literal in literals:
        clauses=literalsDict[literal]
        for clause in clauses:
            delClauses.append(clause)
    for clause in delClauses:
        try:
            del constraints[clause]
        except:
            continue
            
    counterLiterals=[-x for x in literals]
    for literal in counterLiterals:
        try:
            clauses=literalsDict[literal]
        except:
            continue
        for clause in clauses:
            try:
                del constraints[clause][literal]
            except:
                continue
    
    if len(literals)>0: 
        return True,constraints
    else:
        return False,constraints
unitCl(constraintDict,literalsDict)

(True,
 {0: {-1: False, -2: False, 5: False},
  1: {-5: False, 1: False},
  2: {-5: False, 2: False},
  3: {-3: False, 6: False},
  4: {-6: False, 3: False}})

In [273]:
def pureLit(constr,litDict):
    constraints=deepcopy(constr)

    foundOne=False
    literals=[]
    for clause in constraints.keys():
        for literal in constraints[clause].keys():
            literals.append(literal)    
    doubleLiterals=[]
    counterLiterals=[-x for x in literals]
    for i in literals:
        for j in counterLiterals:
            if i==j:
                doubleLiterals.append(i)
    singleLiterals=[x for x in literals if x not in doubleLiterals]
    
#     print(singleLiterals,litDict)
#     print(singleLiterals)
#     print(doubleLiterals)
#     print(singleLiterals)
    
    for literal in singleLiterals:
        for clause in litDict[literal]:
#             print(literal,clause)
            try:
                del constraints[clause]
            except:
                continue
#     print(singleLiterals)  
    if len(singleLiterals)>0:
        foundOne=True
    
    counterLiterals=[-x for x in singleLiterals]   
    for literal in counterLiterals:
        try:
            clauses=litDict[literal]
        except:
            continue
        for clause in clauses:
            try:
                del constraints[clause][literal]
            except:
                continue
    return foundOne,constraints
f,c=pureLit(constraintDict,literalsDict)
print(constraintDict)
print(c)
f,c=pureLit(c,literalsDict)
print(c)

{0: {-1: False, -2: False, 5: False}, 1: {-5: False, 1: False}, 2: {-5: False, 2: False}, 3: {-3: False, 6: False}, 4: {-6: False, 3: False}, 5: {9: False}}
{0: {-1: False, -2: False, 5: False}, 1: {-5: False, 1: False}, 2: {-5: False, 2: False}, 3: {-3: False, 6: False}, 4: {-6: False, 3: False}}
{0: {-1: False, -2: False, 5: False}, 1: {-5: False, 1: False}, 2: {-5: False, 2: False}, 3: {-3: False, 6: False}, 4: {-6: False, 3: False}}


In [291]:
def pickRandomLiteral(constr,litDict,prevLit):
    constraints=deepcopy(constr)
#     print(constraints)

    literals=[]
    if not prevLit:
        for clause in constraints.keys():
            for literal in constraints[clause].keys():
                literals.append(literal)
        literal=random.choice(list(set(literals)))
    else:
        literal=prevLit

    clauses=litDict[literal]
#     print(clauses)
    for clause in clauses:
        try:
            del constraints[clause]
        except:
            continue
    counterLiteral=-literal
    try:
        clauses=litDict[counterLiteral]
    except:
        return constraints
    for clause in clauses:
        try:
            del constraints[clause][counterLiteral]
        except:
            continue
    return constraints,literal
pickRandomLiteral(constraintDict,literalsDict,None)

({0: {-1: False, -2: False, 5: False},
  1: {-5: False, 1: False},
  2: {-5: False, 2: False},
  4: {-6: False},
  5: {9: False}},
 -3)